In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_raw=pd.read_csv('../content/water_potability.csv')
#df_raw=pd.read_csv('water_potability.csv')

# drop Nan data and reset index
df_dropped = df_raw.dropna()
df = df_dropped.reset_index(drop=True)
print("raw data has", len(df_raw), " after drop has",  len(df))

raw data has 3276  after drop has 2011


In [ ]:
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
1,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
2,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
3,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
4,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0


In [ ]:
X = df.drop('Potability',axis=1).values
y = df['Potability'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# import the class for LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc,roc_curve

In [ ]:
# Cross-Valiation cv=5 and 10 on training set only for Combination of penalty and solver
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

clf = [
    LogisticRegression(solver='liblinear',penalty='l1'),
    LogisticRegression(solver='saga',penalty='l1', max_iter=10000),
    LogisticRegression(solver='lbfgs',penalty='l2', max_iter=10000),
    LogisticRegression(solver='liblinear',penalty='l2'),
    LogisticRegression(solver='newton-cholesky',penalty='l2', max_iter=10000),
    LogisticRegression(solver='sag',penalty='l2', max_iter=10000),
    LogisticRegression(solver='saga',penalty='l2', max_iter=10000)
  ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)


row_index = 0
for alg in clf:
    # cross-validation on training dataset
    score5_train = cross_val_score(alg, X_train, y_train, cv=10)


    predicted = alg.fit(X_train, y_train).predict(X_test)
    fp, tp, th = roc_curve(y_test, predicted)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Cross-Validation Accuracy'] = round(np.mean(score5_train), 5)
    clf_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(X_train, y_train), 5)
    clf_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(X_test, y_test), 5)
    clf_compare.loc[row_index, 'Precision'] = round(precision_score(y_test, predicted),5)
    clf_compare.loc[row_index, 'Recall'] = round(recall_score(y_test, predicted),5)
    clf_compare.loc[row_index, 'AUC'] = round(auc(fp, tp),5)

    row_index+=1

clf_compare

,Cross-Validation Accuracy,Train Accuracy,Test Accuracy,Precision,Recall,AUC
0,0.59774,0.60199,0.58444,0.75,0.01186,0.5045
1,0.59774,0.60057,0.58444,0.75,0.01186,0.5045
2,0.59774,0.60128,0.58113,0.50,0.00791,0.5011
3,0.59774,0.60199,0.58113,0.50,0.00791,0.5011
4,0.59774,0.60128,0.58113,0.50,0.00791,0.5011
5,0.59774,0.60128,0.58113,0.50,0.00791,0.5011
6,0.59774,0.60128,0.58113,0.50,0.00791,0.5011
